In [2]:
import tiktoken 

In [3]:
encoder = tiktoken.encoding_for_model('gpt-4')  
x = encoder.encode('hi there this is naman jaiN') 
x

[6151, 1070, 420, 374, 308, 13005, 503, 2192, 45]

In [4]:
with open('input.txt', 'r', encoding = 'utf-8') as f: 
    text = f.read()
print(len(text))
print(text[:10])

1115394
First Citi


In [5]:
chars = sorted(set(list(text))) 
stoi =  {i : ch for ch, i in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(itos)
#encode = lambda s: [stoi[char] for char in s]
encode = lambda s: [stoi[c] for c in s] 
decode = lambda s: ''.join([itos[ix] for ix in s])
print(vocab_size)
print(encode("hi there"))
print(decode(encode('naman jain')))


65
[46, 47, 1, 58, 46, 43, 56, 43]
naman jain


In [6]:
import torch 
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:40])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56])


In [7]:
n = int(0.9 * data.shape[0])
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [8]:
block_size = 8 
context_length = 8 
train_data[:context_length +1] 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:context_length]
y = train_data[1:context_length+1] 
for i in range(context_length): 
    context = x[:i+1]
    target = y[i]
    print(f'For the input data {context} output is {target}')

For the input data tensor([18]) output is 47
For the input data tensor([18, 47]) output is 56
For the input data tensor([18, 47, 56]) output is 57
For the input data tensor([18, 47, 56, 57]) output is 58
For the input data tensor([18, 47, 56, 57, 58]) output is 1
For the input data tensor([18, 47, 56, 57, 58,  1]) output is 15
For the input data tensor([18, 47, 56, 57, 58,  1, 15]) output is 47
For the input data tensor([18, 47, 56, 57, 58,  1, 15, 47]) output is 58


In [10]:
torch.random.manual_seed(1337) 
batch_size = 4 
def get_batch(split): 
    data = {
        'train': train_data, 
        'val': val_data,
    }[split]
    ix = torch.randint(0, len(data) - context_length, (batch_size,)) 
    x = torch.stack([data[i: i+context_length] for i in ix])
    y = torch.stack([data[i+1: i+context_length+1] for i in ix])
    return x,y 

xb,yb = get_batch('train') 
print(xb.shape, yb.shape)
print(xb)
print(yb)
for i in range(batch_size): 
    for j in range(context_length): 
        x1 = xb[i,:j+1]
        y1 = yb[i,j]
        print(f'for the input {x1}, the output is {y1}')

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
for the input tensor([24]), the output is 43
for the input tensor([24, 43]), the output is 58
for the input tensor([24, 43, 58]), the output is 5
for the input tensor([24, 43, 58,  5]), the output is 57
for the input tensor([24, 43, 58,  5, 57]), the output is 1
for the input tensor([24, 43, 58,  5, 57,  1]), the output is 46
for the input tensor([24, 43, 58,  5, 57,  1, 46]), the output is 43
for the input tensor([24, 43, 58,  5, 57,  1, 46, 43]), the output is 39
for the input tensor([44]), the output is 53
for the input tensor([44, 53]), the output is 56
for the input tensor([44, 53, 56]), the output is 1
for the 

In [11]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [12]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337) 


# implement biagram 
class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size): 
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target = None): 
        logits = self.token_embedding_table(idx) 
        if target is None: 
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B * T, -1) 
            target = target.view(-1) 
            loss = F.cross_entropy(logits, target) 
        return logits, loss 

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens): 
            logits, loss = self(idx) 
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1) 
            idx_next = torch.multinomial(probs, num_samples = 1) 
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx 

m = BiagramLanguageModel(vocab_size)
logits, loss = m(xb, yb) 
print(logits.shape)
print(loss) 

print(decode(m.generate(idx = torch.ones((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
 -KIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
for i in range(10000):
    xb,yb = get_batch('train') 
    logits, loss = m(xb,yb) 
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.382369041442871


In [26]:
torch.manual_seed(1337) 
B, T, C = 4,8,2 
xori = torch.randn((B,T,C))

In [31]:
xbow = torch.zeros(B,T,C) 
for b in range(B):
    for t in range(T): 
        xprev = xori[b,:t+1]
        xbow[b,t]= xprev.mean(0) 
print(xbow.shape)

torch.Size([4, 8, 2])


In [34]:
wei = torch.tril(torch.ones(T, T)) 
wei = wei / wei.sum(1, keepdims = True) 
print(xori.shape)
print(wei.shape)
xbow2 = wei @ xori # (B, T, T) @ (B, T, C) -> B, T, C
torch.allclose(xbow, xbow2) 

torch.Size([4, 8, 2])
torch.Size([8, 8])


True

In [38]:
tril = torch.tril(torch.ones(T,T)) 
wei2 = torch. ((T,T)) 
wei2 = wei2.masked_fill(tril == 0, float('-inf'))
wei2 = torch.softmax(wei2, 1) 
xbow2 = wei2 @ xori 
torch.allclose(xbow, xbow2) 

True

In [51]:
B, T, C = 4, 8, 32 
xori1 = torch.randn(B,T,C) 
head_size = 16
query = nn.Linear(C, head_size, bias = False)
key = nn.Linear(C, head_size, bias = False) 
value = nn.Linear(C, head_size, bias = False) 
k = key(xori1) # B, T, H
q = query(xori1) # B, T, H
wei4 = k @ q.transpose(-1,-2) * head_size**-0.5 # B , T , T 

tril =  torch.tril(torch.ones(T , T))
wei4 = torch.masked_fill(wei4, tril == 0, float('-inf')) 
wei4 = F.softmax(wei4, dim = -1) 

v = value(xori1) 
out = wei4 @ v

out.shape

torch.Size([4, 8, 16])